In [50]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import time

from catboost import CatBoostRegressor

import re

from statsmodels.stats.outliers_influence import variance_inflation_factor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVR

import pickle

pd.set_option('display.max_columns', 100)

In [2]:
def dummify(df, non_dummies, dummies):
    for dummified in dummies:
        for original in non_dummies:
            if original in dummified:
                orig_name = f'{original}_'
                value = dummified.replace(orig_name, '')
                df[dummified] = df[original].map(lambda x: 1 if x == value else 0)
    df=df.drop(non_dummies,axis=1)
    return df

In [17]:
dummies_linear = [   
    'Neighborhood_Blueste',
    'Neighborhood_BrDale',
    'Neighborhood_BrkSide',
    'Neighborhood_ClearCr',
    'Neighborhood_CollgCr',
    'Neighborhood_Crawfor',
    'Neighborhood_Edwards',
    'Neighborhood_Gilbert', 
    'Neighborhood_Greens', 
    'Neighborhood_GrnHill',
    'Neighborhood_IDOTRR',
    'Neighborhood_Landmrk',
    'Neighborhood_MeadowV',
    'Neighborhood_Mitchel',
    'Neighborhood_NAmes',
    'Neighborhood_NPkVill',
    'Neighborhood_NWAmes',
    'Neighborhood_NoRidge',
    'Neighborhood_NridgHt',
    'Neighborhood_OldTown',
    'Neighborhood_SWISU',
    'Neighborhood_Sawyer',
    'Neighborhood_SawyerW',
    'Neighborhood_Somerst',
    'Neighborhood_StoneBr',
    'Neighborhood_Timber',
    'Neighborhood_Veenker',
    'BldgType_2fmCon',
    'BldgType_Duplex',
    'BldgType_Twnhs',
    'BldgType_TwnhsE',
    'MasVnrType_None',
    'MasVnrType_Stone',
    'BSMT_HighQual_bin_500-1000',
    'BSMT_HighQual_bin_0-500',
    'BSMT_HighQual_bin_1000-1500',
    'BSMT_HighQual_bin_1500+',
    'BSMT_LowQual_bin_0-500',
    'BSMT_LowQual_bin_500-1000',
    'BSMT_LowQual_bin_1000-1500',
    'BSMT_LowQual_bin_1500+'
    ]

non_dummies_linear = ['Neighborhood', 'BldgType', 'MasVnrType', 'BSMT_HighQual_bin', 'BSMT_LowQual_bin']

In [43]:
back_end_linear = front_end.copy()
back_end_linear.drop(columns = ['PID', 'SalePrice'], axis =1, inplace = True)
back_end_linear['GrLivArea_log'] = np.log10(back_end_linear['GrLivArea'])
back_end_linear['LotArea_log'] = np.log10(back_end_linear['LotArea'])
back_end_linear['ExterQualDisc'] = back_end_linear['ExterQual'] - back_end_linear['OverallQual']
back_end_linear['OverallCondDisc'] = back_end_linear['OverallCond'] - back_end_linear['OverallQual']
back_end_linear['KitchenQualDisc'] = back_end_linear['KitchenQual'] - back_end_linear['OverallQual']
back_end_linear = back_end_linear.drop(['ExterQual','OverallCond','KitchenQual'], axis=1)

back_end_linear['BSMT_LowQual_bin'] = pd.cut(back_end_linear['BSMT_LowQual'], [-1, 1, 500, 1000, 1500, 2500], labels = ['No basement', '0-500', '500-1000', '1000-1500', '1500+']) 
back_end_linear['BSMT_HighQual_bin'] = pd.cut(back_end_linear['BSMT_HighQual'], [-1, 1, 500, 1000, 1500, 2500], labels = ['No basement', '0-500', '500-1000', '1000-1500', '1500+']) 
back_end_linear.drop(['BSMT_HighQual', 'BSMT_LowQual', 'GrLivArea', 'LotArea'], axis = 1, inplace = True)

back_end_linear = dummify(back_end_linear, non_dummies_linear, dummies_linear)

back_end_svr = back_end_linear.copy()


In [44]:
back_end_svr = pd.DataFrame(svr_backend_scaler.fit_transform(back_end_svr), columns = back_end_linear.columns)


In [21]:
with open('../Matt/SVR_model.pickle', mode = 'rb') as file:
    svr = pickle.load(file)

/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVR from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [46]:
svr.get_params()

{'C': 100,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [6]:
svr_backend_scaler = StandardScaler()
svr_price_scaler = StandardScaler()

In [42]:
housing_coords = pd.read_csv('../data/ames_housing_price_data_v6.csv', index_col = 0);
housing_coords.drop("SalePrice.1", axis = 1, inplace = True);

y = housing_coords["SalePrice"]
y_std = svr_price_scaler.fit_transform(np.array(np.log10(y)).reshape(-1,1));
front_end = housing_coords.copy();
front_end.reset_index(inplace = True)

In [45]:
svr.score(back_end_svr, y_std)

0.9387299448505945

### What do I want to do on SVR Gaussian kernel?
- Nominal: dummify 
- Ordinal: keep
- Discrete: keep
- Continuous: keep
- Then, StandardScaler it.

Parameters to be tuned:
- gamma: large=overfit, small=underfit
- C: large=overfit, small=underfit
- epsilon: large=underfit, small=overfit

In [51]:
back_end_linear.head() # BSMT: no bin

,OverallQual,house_age_years,GarageCars,FullBath,HalfBath,BsmtExposure_ord,SaleTypeNew,PorchSF,Fireplaces,Pool,BedroomAbvGr,water_tower,graveyard,police,optician,slipway,bar,cinema,supermarket,hotel,stop,farmyard,christian_catholic,jewish,muslim,garden_centre,christian_lutheran,GrLivArea_log,LotArea_log,ExterQualDisc,OverallCondDisc,KitchenQualDisc,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,MasVnrType_None,MasVnrType_Stone,BSMT_HighQual_bin_500-1000,BSMT_HighQual_bin_0-500,BSMT_HighQual_bin_1000-1500,BSMT_HighQual_bin_1500+,BSMT_LowQual_bin_0-500,BSMT_LowQual_bin_500-1000,BSMT_LowQual_bin_1000-1500,BSMT_LowQual_bin_1500+
0,0.428571,71.210959,2.0,1,0,1,0,166,1,0,2,2,1,1,1,0,11,0,4,0,0,0,1,0,0,0,3,2.932474,3.897077,-0.095238,0.071429,-0.095238,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,0.285714,25.104110,1.0,2,0,2,0,105,0,0,2,1,1,1,1,0,11,0,4,0,0,0,1,0,1,0,2,3.020775,3.626853,0.380952,0.047619,0.380952,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
2,0.142857,109.402740,1.0,1,0,1,0,279,0,0,2,0,1,1,1,0,4,1,1,0,0,1,0,0,0,0,3,3.016616,3.910944,0.523810,0.690476,0.190476,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
3,0.714286,8.838356,2.0,2,1,1,0,45,0,0,3,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,3,3.221414,3.924279,-0.047619,-0.214286,-0.047619,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0
4,0.571429,6.501370,2.0,3,0,0,0,177,1,0,4,1,0,0,1,0,2,0,4,3,1,0,0,0,0,0,1,3.283753,3.863382,0.095238,-0.238095,0.095238,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
#function and dictionaries required to transform front-end to back-end
dummies = [   
    'Neighborhood_Blueste',
    'Neighborhood_BrDale',
    'Neighborhood_BrkSide',
    'Neighborhood_ClearCr',
    'Neighborhood_CollgCr',
    'Neighborhood_Crawfor',
    'Neighborhood_Edwards',
    'Neighborhood_Gilbert', 
    'Neighborhood_Greens', 
    'Neighborhood_GrnHill',
    'Neighborhood_IDOTRR',
    'Neighborhood_Landmrk',
    'Neighborhood_MeadowV',
    'Neighborhood_Mitchel',
    'Neighborhood_NAmes',
    'Neighborhood_NPkVill',
    'Neighborhood_NWAmes',
    'Neighborhood_NoRidge',
    'Neighborhood_NridgHt',
    'Neighborhood_OldTown',
    'Neighborhood_SWISU',
    'Neighborhood_Sawyer',
    'Neighborhood_SawyerW',
    'Neighborhood_Somerst',
    'Neighborhood_StoneBr',
    'Neighborhood_Timber',
    'Neighborhood_Veenker',
    'BldgType_2fmCon',
    'BldgType_Duplex',
    'BldgType_Twnhs',
    'BldgType_TwnhsE',
    'MasVnrType_None',
    'MasVnrType_Stone'
    ]

non_dummies=['Neighborhood', 'BldgType', 'MasVnrType']

In [54]:
svrg_backend_scaler = StandardScaler()

In [55]:
#transformation of front-end to back-end, and catboost application
back_end = front_end.copy()
back_end.drop(columns = ['PID', 'SalePrice'], axis =1, inplace = True)
back_end['ExterQualDisc']=back_end['ExterQual']-back_end['OverallQual']
back_end['OverallCondDisc']=back_end['OverallCond']-back_end['OverallQual']
back_end['KitchenQualDisc']=back_end['KitchenQual']-back_end['OverallQual']
back_end=back_end.drop(['ExterQual','OverallCond','KitchenQual'],axis=1)

back_end = dummify(back_end, non_dummies, dummies)

back_end['GrLivArea_log'] = np.log10(back_end['GrLivArea'])
back_end['LotArea_log'] = np.log10(back_end['LotArea'])
back_end.drop(['BSMT_HighQual', 'BSMT_LowQual', 'GrLivArea', 'LotArea'], axis = 1, inplace = True)

# Standardize
back_end_svr = pd.DataFrame(svrg_backend_scaler.fit_transform(back_end_svr), columns = back_end_linear.columns)


In [58]:
back_end_svr.shape
y_std.shape

(2455, 1)

In [98]:
kfold = KFold(n_splits=5, shuffle = True)
params = {
    'C': np.logspace(-0.5,1.5,9),
    'epsilon': np.logspace(-2,-1,5),
    'gamma' : np.logspace(-3.5,-2,7)
          }
svr = SVR(kernel = 'rbf')
svr_tuner = GridSearchCV(svr, params, cv=kfold, return_train_score = True, n_jobs = -1)
%time svr_tuner.fit(back_end_svr, y_std.ravel())
# Maybe: C: 0.1-1000, better: 1-5, best:2
# Maybe: epsilon: 0.01-0.1
# Maybe: gamma: 0.0003-0.01

CPU times: user 4.12 s, sys: 364 ms, total: 4.49 s
Wall time: 9min 36s


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=SVR(), n_jobs=-1,
             param_grid={'C': array([ 0.31622777,  0.56234133,  1.        ,  1.77827941,  3.16227766,
        5.62341325, 10.        , 17.7827941 , 31.6227766 ]),
                         'epsilon': array([0.01      , 0.01778279, 0.03162278, 0.05623413, 0.1       ]),
                         'gamma': array([0.00031623, 0.00056234, 0.001     , 0.00177828, 0.00316228,
       0.00562341, 0.01      ])},
             return_train_score=True)

In [99]:
svr_tuner.best_score_
# 0.9404, 1.5, -1, -3
# 0.9402, 1.25, -1.25, -3

0.9402227548289458

In [106]:
svr_tuner.best_estimator_.get_params()["C"]

KeyError: ('C', 'gamma')

### An iterative test
1. Set an accuracy goal: Say 0.9450.
2. Pick a suitable start, and try to approach the goal, by adjusting the step size each time.
3. The closer to the goal, the smaller the step size.

In [110]:
C, gamma, epsilon = (18, 0.001, 0.06);
goal = 0.9450;
parameters = [];

In [112]:
kfold = KFold(n_splits=5, shuffle = True);
svr = SVR(kernel = 'rbf');

round_best = 0.94
for i in range(50):
    step_size = 20*max(0.0005, goal-round_best);
    params = {
    'C': np.linspace((1-step_size)*C, (1+step_size)*C, 5),
    'epsilon': np.linspace((1-step_size)*epsilon, (1+step_size)*epsilon, 5),
    'gamma' : np.linspace((1-step_size)*gamma, (1+step_size)*gamma, 5)
          };
    svr_tuner = GridSearchCV(svr, params, cv=kfold, return_train_score = True, n_jobs = -1)
    %time svr_tuner.fit(back_end_svr, y_std.ravel())
    
    C = svr_tuner.best_estimator_.get_params()["C"]
    epsilon = svr_tuner.best_estimator_.get_params()["epsilon"]
    gamma = svr_tuner.best_estimator_.get_params()["gamma"]
    round_best = svr_tuner.best_score_;
    
    print(C, epsilon, gamma, round_best);
    parameters.append([C, epsilon, gamma, round_best]);


CPU times: user 2.11 s, sys: 124 ms, total: 2.23 s
Wall time: 3min 51s
21.78 0.059399999999999994 0.0008549999999999999 0.9399893109463889
CPU times: user 2.87 s, sys: 142 ms, total: 3.01 s
Wall time: 4min 59s
23.96265615175296 0.06535269859568989 0.0009406827828167482 0.940767691487656
CPU times: user 2.8 s, sys: 139 ms, total: 2.94 s
Wall time: 7min 44s
25.991003223941643 0.07088455424711357 0.0009406827828167482 0.9400462465471495
CPU times: user 2.79 s, sys: 139 ms, total: 2.93 s
Wall time: 7min 32s
24.703473004305167 0.06386166214000552 0.0008474845711484399 0.9403302737790508
CPU times: user 2.23 s, sys: 132 ms, total: 2.36 s
Wall time: 6min 17s
27.010642117039453 0.0608794973579197 0.000768334152673605 0.9400729642686223
CPU times: user 2.05 s, sys: 124 ms, total: 2.17 s
Wall time: 5min 45s
29.6722900938016 0.06387905194582752 0.000768334152673605 0.9395050088120002
CPU times: user 2.41 s, sys: 136 ms, total: 2.55 s
Wall time: 7min 21s
32.933269545665866 0.07089934849662957 0.00

In [117]:
fig = px.scatter();
for i in [3]:
    param = [p[i] for p in parameters];
    fig.add_trace(go.Scatter(x = list(range(50)), y=param));
fig.show();

In [131]:
svr_tuner.cv_results_["mean_test_score"].mean()

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [129]:
parameters2 = [];

In [132]:
kfold = KFold(n_splits=5, shuffle = True);
svr = SVR(kernel = 'rbf', epsilon = 0.1);

round_best = 0.94
for i in range(50):
    step_size = 30*max(0.0005, goal-round_best);
    params = {
    'C': np.linspace((1-step_size)*C, (1+step_size)*C, 7),
    # 'epsilon': np.linspace((1-step_size)*epsilon, (1+step_size)*epsilon, 5),
    'gamma' : np.linspace((1-step_size)*gamma, (1+step_size)*gamma, 7)
          };
    svr_tuner = GridSearchCV(svr, params, cv=kfold, return_train_score = True, n_jobs = -1)
    %time svr_tuner.fit(back_end_svr, y_std.ravel())
    
    C = svr_tuner.best_estimator_.get_params()["C"]
    gamma = svr_tuner.best_estimator_.get_params()["gamma"]
    round_best = svr_tuner.best_score_;
    round_mean = svr_tuner.cv_results_["mean_test_score"].mean();
    
    print(C, gamma, round_best, round_mean);
    parameters2.append([C, gamma, round_best, round_mean]);


CPU times: user 1.58 s, sys: 107 ms, total: 1.68 s
Wall time: 1min 32s
519.7801926854717 0.00019517783297986278 0.941856035365374 0.9417340282201507
CPU times: user 1.59 s, sys: 48.9 ms, total: 1.64 s
Wall time: 1min 34s
568.8053089929374 0.00017676886684931414 0.9412391142734009 0.9411613307750539
CPU times: user 1.69 s, sys: 50.2 ms, total: 1.74 s
Wall time: 1min 40s
632.9816620270964 0.00015682464160209392 0.9412512148530847 0.9412117671663148
CPU times: user 1.94 s, sys: 49.9 ms, total: 1.98 s
Wall time: 1min 44s
704.1690296134033 0.00017446169821534095 0.9412130187545775 0.941081825519729
CPU times: user 1.81 s, sys: 51.5 ms, total: 1.86 s
Wall time: 1min 51s
730.835778700936 0.00015464120283975965 0.9413844769584495 0.9412959805726611
CPU times: user 1.74 s, sys: 52.7 ms, total: 1.79 s
Wall time: 1min 48s
677.9887067512822 0.00015464120283975965 0.9413743009956208 0.941326087291858
CPU times: user 1.87 s, sys: 51.1 ms, total: 1.92 s
Wall time: 1min 45s
751.734196122735 0.00015464

In [133]:
temp = parameters2
temp.sort(key = lambda x: x[3], reverse = True)

In [145]:
(temp[0][1]*temp[1][1]*temp[2][1]*temp[3][1]*temp[4][1])**0.2 # C about 950, gamma about 1.5e-4

0.00014967910984191868

In [154]:
kfold = KFold(n_splits=5, shuffle = True)
params = {
    'C': [950],
    'epsilon': [0.1],
    'gamma' : [1.5e-4]
          }
svr = SVR(kernel = 'rbf')
svr_tuner = GridSearchCV(svr, params, cv=kfold, return_train_score = True)
svr_tuner.fit(back_end_svr, y_std.ravel())

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=SVR(),
             param_grid={'C': [950], 'epsilon': [0.1], 'gamma': [0.00015]},
             return_train_score=True)

In [155]:
svr_tuner.best_score_

0.9405229936873749

In [156]:
with open('SVR_model_kernel.pickle', mode = 'wb') as file:
     pickle.dump(svr_tuner.best_estimator_, file)